# SCC02713 - Introdução à Bioinformática
## Trabalho Prático: Classificação de Elementos Transponíveis
### Alunos:
Marcos Paulo Rocha Baltazar - 13782373

Rebeca Vieira Carvalho - 12543530

Yvis Freire Silva Santos - 12608793





In [ ]:
# Instalação dos prerequisitos
!pip install biopython wget
!git clone https://github.com/Bonidia/MathFeature.git MathFeature
%cd MathFeature
!pip install -r requirements.txt
!apt-get -y install python3-igraph
!pip install scikit-learn
%cd ..

fatal: destination path 'MathFeature' already exists and is not an empty directory.
/home/yvis/dev/bioinfo-final-project/MathFeature
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/home/yvis/dev/bioinfo-final-project


In [ ]:
# Importação das bibliotecas

import os
import wget
import pandas as pd
import time
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from sklearn.model_selection import train_test_split

In [ ]:
# Download automático dos arquivos .gff3 dos elementos transponíveis de cada classe

classes_te = {
    'LTRS': 'TEAnnotationFinal_LTR.gff3',
    'LINES': 'TEAnnotationFinal_LINE.gff3',
    'SINES': 'TEAnnotationFinal_SINE.gff3',
    'TIRS': 'TEAnnotationFinal_TIR.gff3',
    'MITES': 'TEAnnotationFinal_MITE.gff3',
    'Helitrons': 'TEAnnotationFinal_Helitron.gff3'
}

url = 'http://apte.cp.utfpr.edu.br/te-annotation/zea_mays/'

gff_dir = 'gff3_files/'
os.makedirs(gff_dir, exist_ok=True)

for classe, filename in classes_te.items():
    if not os.path.exists(gff_dir + filename):
        wget.download(url + filename, out=gff_dir)

print("Arquivos .gff3 baixados.")

Arquivos .gff3 baixados.


In [ ]:
# Recuperando as colunas 'Chr', 'Start', 'End' das linha com strand positivo

col_names = ['Chr', 'Source', 'Class', 'Start', 'End', 'Score', 'Strand', 'Phase', 'Attributes']

all_te_locations = []

print("Iniciando processamento dos arquivos GFF3...")

for classe, filename in classes_te.items():
    file_path = os.path.join(gff_dir, filename)

    try:
        df = pd.read_csv(
            file_path,
            sep='\t',
            comment='#',
            header=None,
            names=col_names
        )

        # 1. Filtrar linhas com Strand positivo (+)
        df_filtered = df[df['Strand'].astype(str).str.strip() == '+'].copy()

        # 2. Recuperar Chr, Start e End
        df_filtered = df_filtered[['Chr', 'Start', 'End']]
        df_filtered['Start'] = pd.to_numeric(df_filtered['Start'])
        df_filtered['End'] = pd.to_numeric(df_filtered['End'])

        # 3. Adicionar a coluna da Classe
        df_filtered['Classe'] = classe

        all_te_locations.append(df_filtered)
        print(f"  - Processado {filename}: {len(df_filtered)} TEs com Strand (+)")

    except FileNotFoundError:
        print(f"ERRO: Arquivo {file_path} não encontrado. Certifique-se de fazer o upload.")
    except Exception as e:
        print(f"Erro ao processar {filename}: {e}")

# Consolida todos os TEs em um único DataFrame
if all_te_locations:
    df_locations = pd.concat(all_te_locations, ignore_index=True)
    print(f"\nProcessamento concluído. Total de TEs a extrair: {len(df_locations)}")
    print(df_locations.head())
else:
    print("\nNenhum arquivo GFF processado. Verifique os uploads e nomes dos arquivos.")

Iniciando processamento dos arquivos GFF3...
  - Processado TEAnnotationFinal_LTR.gff3: 191815 TEs com Strand (+)
  - Processado TEAnnotationFinal_LINE.gff3: 11174 TEs com Strand (+)
  - Processado TEAnnotationFinal_SINE.gff3: 3755 TEs com Strand (+)
  - Processado TEAnnotationFinal_TIR.gff3: 83849 TEs com Strand (+)
  - Processado TEAnnotationFinal_MITE.gff3: 26350 TEs com Strand (+)
  - Processado TEAnnotationFinal_Helitron.gff3: 24769 TEs com Strand (+)

Processamento concluído. Total de TEs a extrair: 341712
  Chr      Start        End Classe
0   4  122450121  122450632   LTRS
1   5    4778869    4779015   LTRS
2   2  197296548  197297070   LTRS
3   9   89034632   89035827   LTRS
4   9   39829863   39830128   LTRS


In [ ]:
# Download automático do Genoma (arquivos .fasta) via NCBI

Entrez.email = "yvis_freire@usp.br"

accession_numbers = {
    '1': 'LR618874.1',
    '2': 'LR618875.1',
    '3': 'LR618876.1',
    '4': 'LR618877.1',
    '5': 'LR618878.1',
    '6': 'LR618879.1',
    '7': 'LR618880.1',
    '8': 'LR618881.1',
    '9': 'LR618882.1',
    '10': 'LR618883.1',
    'MT': 'AY506529.1',
    'Pitd': 'X86563.2'
}

fasta_dir = 'fasta_files/'
os.makedirs(fasta_dir, exist_ok=True)

print("Baixando sequências do genoma...")

genomes = {}

for chr_name, acc_num in accession_numbers.items():
    fasta_path = os.path.join(fasta_dir, f"chr_{chr_name}.fasta")

    if not os.path.exists(fasta_path):
        try:
            handle = Entrez.efetch(db="nucleotide", id=acc_num, rettype="fasta", retmode="text")
            fasta_data = handle.read()
            handle.close()

            with open(fasta_path, "w") as f:
                f.write(fasta_data)
            print(f"  - Cromossomo {chr_name} ({acc_num}) baixado.")

        except Exception as e:
            print(f"Erro ao baixar Chr {chr_name}: {e}")
    else:
        print(f"  - Cromossomo {chr_name} já existe, pulando download.")

    try:
        record = SeqIO.read(fasta_path, "fasta")
        genomes[chr_name] = record.seq
    except Exception as e:
        print(f"Erro ao ler FASTA para Chr {chr_name}: {e}")

print("\nSequências do genoma carregadas na memória.")

Baixando sequências do genoma...
  - Cromossomo 1 já existe, pulando download.
  - Cromossomo 2 já existe, pulando download.
  - Cromossomo 3 já existe, pulando download.
  - Cromossomo 4 já existe, pulando download.
  - Cromossomo 5 já existe, pulando download.
  - Cromossomo 6 já existe, pulando download.
  - Cromossomo 7 já existe, pulando download.
  - Cromossomo 8 já existe, pulando download.
  - Cromossomo 9 já existe, pulando download.
  - Cromossomo 10 já existe, pulando download.
  - Cromossomo MT já existe, pulando download.
  - Cromossomo Pitd já existe, pulando download.

Sequências do genoma carregadas na memória.


In [38]:
# Extração dos elementos transponíveis

print("Iniciando extração das sequências de TEs...")

seq_dataset = []

for index, row in df_locations.iterrows():
    chr_name = str(row['Chr'])
    start = int(row['Start'])
    end = int(row['End'])
    classe = row['Classe']

    chromosome_seq = genomes.get(chr_name)

    if chromosome_seq:
        # Ajuste de índice: GFF (1-based) para Python (0-based)
        start_idx = start - 1
        end_idx = end

        te_seq = chromosome_seq[start_idx:end_idx]

        seq_dataset.append({
            'Cromossomo': chr_name,
            'Sequencia': str(te_seq),
            'Classe': classe
        })

df_final = pd.DataFrame(seq_dataset)


if df_final.empty:
    print("ERRO: Nenhum dado de TE foi extraído.")
else:
    print(f"\nExtração concluída. Total de sequências: {len(df_final)}")
    print("Dataset final (amostra):")
    print(df_final.head())

TAMANHO_AMOSTRA = 10000
total_sequencias = len(df_final)

# Verifica se o dataframe é maior que a amostra desejada
if total_sequencias > TAMANHO_AMOSTRA:
    print(f"\nTotal de {total_sequencias} sequências. Amostrando {TAMANHO_AMOSTRA} (mantendo proporção)...")

    # Extração de uma amostra balanceada da base de dados
    df_amostra, _ = train_test_split(
        df_final,
        train_size=TAMANHO_AMOSTRA,
        shuffle=True,
        stratify=df_final['Classe'],
        random_state=42
    )

    df_amostra = df_amostra.drop(125649)
    df_amostra = df_amostra.drop(884)
    df_amostra = df_amostra.drop(184488)

    # Salva a amostra estratificada
    df_amostra.to_csv('te_dataset_final.csv', index=False)
    print(f"Salvo {len(df_amostra)} sequências estratificadas em 'te_dataset_final.csv'")

    print("\nProporções no Dataset Original (Completo):")
    print(df_final['Classe'].value_counts(normalize=True))
    
    print("\nProporções na Amostra de 10000:")
    print(df_amostra['Classe'].value_counts(normalize=True))

else:
    # Se o total for 10000 ou menos, apenas embaralha e salva tudo
    print(f"\nTotal de {total_sequencias} sequências (menor ou igual a {TAMANHO_AMOSTRA}). Embaralhando e salvando todas.")
    
    df_final_embaralhado = df_final.sample(frac=1).reset_index(drop=True)
    df_final_embaralhado.to_csv('te_dataset_final.csv', index=False)
    print(f"Salvo {len(df_final_embaralhado)} sequências em 'te_dataset_final.csv'")

Iniciando extração das sequências de TEs...

Extração concluída. Total de sequências: 338334
Dataset final (amostra):
  Cromossomo                                          Sequencia Classe
0          4  TCACAAAGTGGATAGAATACAAGCACATCACCTCTTTGACCTCAGC...   LTRS
1          5  ACCAATTGTGTGGTGGCCCTTGCGGGGAAGTTTTGTTCCCGGCTTT...   LTRS
2          2  GCCGTTGGCGCGACGCGGGGCGCGCGCGGGCGTGCTCTGGCGAGCT...   LTRS
3          9  GTCCGGGGGACCGTTGTCCGCCCCGAGGCTATTTTAGCCAGCTCAT...   LTRS
4          9  GAAGCAGCTCGTGGAGATGCTCAAGCGGTCACCCTACTTCCTCTCC...   LTRS

Total de 338334 sequências. Amostrando 10000 (mantendo proporção)...
Salvo 9997 sequências estratificadas em 'te_dataset_final.csv'

Proporções no Dataset Original (Completo):
Classe
LTRS         0.561058
TIRS         0.245692
MITES        0.077175
Helitrons    0.072461
LINES        0.032707
SINES        0.010906
Name: proportion, dtype: float64

Proporções na Amostra de 10000:
Classe
LTRS         0.560868
TIRS         0.245774
MITES        0.077223
H

In [ ]:
for idx, row in df_amostra.iterrows():
    row['id'] = idx

df_amostra.head()
df_amostra.to_csv('te_dataset_final_index.csv')

In [ ]:
# Salvando arquivo fasta com as TEs

import subprocess

resultados = []
registros = []



with open('meu_arquivo.txt', 'w', encoding='utf-8') as f:
    for idx, row in df_amostra.iterrows():
        reg = SeqRecord(Seq(row["Sequencia"]), id=f"seq_{idx}", description=row["Classe"])
        f.write(f"{idx}\n")
        registros.append(reg)

SeqIO.write(registros, "unprocessed_todas.fasta", "fasta")


9997

In [40]:
# Preprocessamento do arquivo com as sequencias com ferramenta do MathFeature
cmd = [
    "python3", "MathFeature/preprocessing/preprocessing.py",
    "-i", "unprocessed_todas.fasta",
    "-o", "todas.fasta"
]

try:
    subprocess.run(cmd, check=True, capture_output=True, text=True)
except subprocess.CalledProcessError as e:
    print("=== ERRO NO SCRIPT MATHFEATURE ===")
    print(e.stdout)
    print(e.stderr)

In [ ]:
# Rodar MathFeature no modo Kmers
cmd = [
    "python3", "MathFeature/methods/ExtractionTechniques.py",
    "-i", "todas.fasta",
    "-o", "features_kmers.csv",
    "-l", "DNA",
    "-t", "kmer",
    "-seq", "1"
]

try:
    subprocess.run(cmd, input='3\n',check=True, capture_output=True, text=True)
except subprocess.CalledProcessError as e:
    print("=== ERRO NO SCRIPT MATHFEATURE ===")
    print(e.stdout)
    print(e.stderr)

# Ler features geradas
df_feat = pd.read_csv("features_kmers.csv")

df_feat.head()

In [ ]:
# Rodar MathFeature no modo de mapeamento numérico e transformada de Fourier
cmd = [
    "python3", "MathFeature/methods/ExtractionTechniques.py",
    "-i", "todas.fasta",
    "-o", "features_fourier.csv",
    "-l", "DNA",
    "-t", "kmer",
    "-seq", "1"
]

try:
    subprocess.run(cmd, input='3\n',check=True, capture_output=True, text=True)
except subprocess.CalledProcessError as e:
    print("=== ERRO NO SCRIPT MATHFEATURE ===")
    print(e.stdout)
    print(e.stderr)

# Ler features geradas
df_feat = pd.read_csv("features_fourier.csv")

df_feat.head()